In [19]:
import json
import re

from utils import query

## Build Lookup Files

In [21]:
db_file = '../src/data/mimic_iii_processed/mimic.db'
model = query(db_file)
(db_meta, db_tabs, db_head) = model._load_db(db_file)


for tb in db_meta:
    for hd in db_meta[tb]:
        mysql = 'select distinct {} from {}'.format(hd, tb)
        myres = model.execute_sql(mysql).fetchall()
        myres = list({k[0]: {} for k in myres if not k[0] == None})
        db_meta[tb][hd] = myres

## Logic Form Accuracy:

In [25]:
headerDic = []
for tb in db_meta:
    for hd in db_meta[tb]:
        headerDic.append('.'.join([tb,hd]).lower())

In [50]:
cnt = 0
sql_rec = []
generated_sql_file = open('../src/data/generated-sql/predicted-sql.json', 'r')

for line in generated_sql_file:
    line = json.loads(line)
    pred = re.split('<stop>', line['pred_sql'])[0]
    real_sql = line['input_sql']

    predArr = re.split("where", pred)
    predAgg = re.split("\s", predArr[0])
    predAgg = list(filter(None, predAgg))
    predAgg2 = []
    for k in range(len(predAgg)-1):
        if predAgg[k] in headerDic and predAgg[k+1] in headerDic:
            predAgg2.append(predAgg[k] + ',')
        else:
            predAgg2.append(predAgg[k])
    predAgg2.append(predAgg[-1])
    predAgg = ' '.join(predAgg2)
    
    predCon = re.split("and", predArr[1])
    predConNew = []
    k = 0
    while k < len(predCon):
        if "=" in predCon[k] or "<" in predCon[k] or ">" in predCon[k] or ">" in predCon[k]:
            predConNew.append(predCon[k])
        else:
            if(len(predConNew) > 0):
                predConNew[-1] += " and " + predCon[k]
            k += 1
        k += 1
    for k in range(len(predConNew)):
        if "=" in predConNew[k]:
            conOp = "="
        if ">" in predConNew[k]:
            conOp = ">"
        if "<" in predConNew[k]:
            conOp = "<"
        if "<=" in predConNew[k]:
            conOp = "<="
        if ">=" in predConNew[k]:
            conOp = ">="
        conVal = re.split("=|<|>", predConNew[k])
        conVal = list(filter(None, conVal))
        conCol = conVal[0]
        conColArr = re.split('\.|\s', conCol)
        conColArr = list(filter(None, conColArr))
        try:
            pool_ = db_meta[conColArr[0].upper()][conColArr[1].upper()]
        except:
            sql_rec.append(["Error", real_sql])
            continue
        conVal = re.split('"|\s', conVal[-1])
        conVal = list(filter(None, conVal))
        conVal = ' '.join(conVal)
        predConNew[k] = conCol + conOp + ' "' + conVal + '"'

    pred = predAgg + ' where ' + ' and '.join(predConNew)
    pred = re.split("\s", pred)
    pred = list(filter(None, pred))
    pred = " ".join(pred)

    sql_rec.append([pred, real_sql])
    try:
        myres = model.execute_sql(pred).fetchall()
        myres = list({k[0]: {} for k in myres if not k[0] == None})
        cnt += 1
    except:
        pass
    
generated_sql_file.close()

In [60]:
correct = 0
for itm in sql_rec:
    # arr = re.split(',|\s', itm[0].lower())
    # arr = list(filter(None, arr))
    # arr = ' '.join(arr)

    if itm[0] == itm[1]:
        correct += 1
print(f'Logic Form Accuracy: {correct/1000}')

Logic Form Accuracy: 0.442
